In [53]:
#from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Laboras3').getOrCreate()


from pyspark.sql.functions import udf, log, col
from pyspark.sql.types import *
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

#nuo spark 3.0 
#spark.conf.set("spark.sql.legacy.setCommandRejectsSparkCoreConfs","false")

#spark.conf.set("spark.dynamicAllocation.enabled", "true")
#spark.conf.set("spark.executor.instances", 4)
#spark.conf.set("spark.executor.cores", 4)

text_file = spark.sparkContext.textFile("duom_cut.txt")
#text_file = sc.textFile("hdfs:///user/stud0/labt/duom_cut.txt")

In [47]:
def parsinam(line):
	#line=line[2:len(line)-2]
	return line[2:len(line)-2].split('}}{{')

def parsinam2(line):
	objs = line.split('}{')
	marsrutas=None
	sustojimo_data=None
	masinos_tipas=None
	svoris=None
 
	for at in objs:
		temp = at.split('=')
		if(len(temp)<2): 
			break
		key,val=at.split('=')
		if(key == 'marsrutas'):
			marsrutas=val
		if(key == 'sustojimo data'):
			sustojimo_data=val
		if(key == 'Masinos tipas'):
			masinos_tipas=val
		if(key == 'svoris'):
			svoris=val
	if(marsrutas!=None and sustojimo_data!=None and masinos_tipas!=None and svoris!=None):
		return ((marsrutas+"_"+sustojimo_data, (masinos_tipas, float(svoris))))
	#else:
	#	return (0,(0,0))

In [48]:
fmap = text_file.flatMap(parsinam)
p = fmap.map(parsinam2).filter(lambda x: x != None)

In [49]:
mmap = p.reduceByKey(lambda a,b: (a[0], a[1]+b[1]))
mmap.collect()

[('102_2018-01-02', ('van', 402.8)),
 ('102_2018-01-03', ('van', 1363.7)),
 ('102_2018-01-05', ('van', 574.95)),
 ('102_2018-01-10', ('van', 868.3)),
 ('102_2018-01-16', ('van', 1210.5499999999997)),
 ('102_2018-01-17', ('van', 1288.7000000000003)),
 ('102_2018-01-18', ('van', 899.0000000000001)),
 ('102_2018-01-19', ('van', 1052.7500000000002)),
 ('102_2018-01-22', ('van', 1096.2)),
 ('102_2018-01-23', ('van', 374.8)),
 ('102_2018-01-24', ('van', 1287.6)),
 ('102_2018-01-25', ('van', 829.5)),
 ('102_2018-01-29', ('van', 170.85)),
 ('103_2018-01-18', ('van', 642.9)),
 ('103_2018-01-19', ('van', 497.15)),
 ('103_2018-01-23', ('van', 1077.5)),
 ('103_2018-01-24', ('van', 924.0)),
 ('102_2018-01-04', ('van', 798.8500000000001)),
 ('102_2018-01-08', ('van', 561.05)),
 ('102_2018-01-09', ('van', 1633.3499999999997)),
 ('102_2018-01-11', ('van', 1061.8)),
 ('102_2018-01-12', ('van', 335.59999999999997)),
 ('102_2018-01-15', ('van', 1031.8000000000002)),
 ('102_2018-01-26', ('van', 725.199999

In [50]:
tipai = p.map(lambda x: (x[1][0])).distinct()
tipai.collect()

['van']

In [52]:
routes = spark.read.option("header",True).csv("RouteSummary.txt")
routes = routes.drop("M", "BendrasAtstumas","BendrasLaikas","BendraKaina")

def makeID(str1, str2):
    return str1+"_"+str2
	
makeID_UDF = udf(lambda z1,z2: makeID(z1,z2),StringType())

routes = routes.withColumn('ID', makeID_UDF("marsrutas", "sustojimo data")).drop("marsrutas", "sustojimo data")
routes = routes.withColumn('BendrasSvoris', col('BendrasSvoris').cast(FloatType()))
routes = routes.filter(col('BendrasSvoris').isNotNull())

In [55]:
for tipas in tipai.collect():
    print('Masinos tipas - ' + tipas)
    
    #filter by type
    filteredDt = mmap.filter(lambda x: x[1][0] == tipas).map(lambda x: (x[0], x[1][1]))
    
    dt = filteredDt.toDF(["ID", "Svoris"]) # RDD -> DataFrame
    joinedDt = dt.join(routes, "ID") #join datasets
    
    assembler = VectorAssembler(inputCols=["Svoris"], outputCol="features")
    
    asdt = assembler.transform(joinedDt)
    asdt = asdt.drop('Svoris').drop('ID')
    
    lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, featuresCol='features', labelCol='BendrasSvoris')
    lrModel = lr.fit(asdt)

    # Print the coefficients and intercept for linear regression
    print("Coefficients: %s" % str(lrModel.coefficients))
    print("Intercept: %s" % str(lrModel.intercept))

    # Summarize the model over the training set and print out some metrics
    trainingSummary = lrModel.summary
    print("numIterations: %d" % trainingSummary.totalIterations)
    print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
    trainingSummary.residuals.show()
    print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
    print("r2: %f" % trainingSummary.r2)
    
    pandasDF = asdt.toPandas()
    pandasDF.head()

    labels = pandasDF['BendrasSvoris'].to_list()
    values = pandasDF['features'].to_list()
    print(labels)
    print(values)

Masinos tipas - van
Coefficients: [0.5496713621023197]
Intercept: 581.0544722931924
numIterations: 2
objectiveHistory: [0.5, 0.4359285682428404, 0.27999712856354014]
+-------------------+
|          residuals|
+-------------------+
|  -277.162109155038|
|  88.05864237974924|
| -322.1380097268898|
|-192.88410379960533|
|-0.2591885142803676|
|  40.08404336554804|
|  42.49092434869726|
|  57.60895730827792|
| -87.40426825788018|
|  513.6286523627332|
|  234.2886574497984|
|-107.50686715706661|
| -84.16583671540491|
| -128.0381665747111|
|-217.17356554829803|
|-39.625413786566696|
|  -97.4508108757359|
|   516.190535677307|
| 145.45243441336368|
|  211.2397840029214|
+-------------------+
only showing top 20 rows

RMSE: 221.252798
r2: 0.441171
[525.2999877929688, 1418.699951171875, 574.9500122070312, 865.4500122070312, 1246.199951171875, 1329.5, 1117.699951171875, 1217.3299560546875, 1096.199951171875, 1300.699951171875, 1523.0999755859375, 929.5, 590.7999877929688, 806.4000244140625, 637.